# Notebook 02: Feature Extraction avec ResNet

Ce notebook démontre l'extraction de features à partir d'images IRM cérébrales en utilisant un modèle ResNet50 pré-entraîné.

## Objectifs
1. Charger et pré-traiter les images IRM
2. Extraire des features avec ResNet50
3. Visualiser les embeddings dans un espace réduit
4. Préparer les données pour le clustering

In [ ]:
# Importations
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import torch
import torchvision.transforms as transforms
from PIL import Image

from src.model.features import FeatureExtractor
from src.model.preprocessing import ImagePreprocessor
from src.data.loader import DataLoaderWrapper

## 1. Configuration et chargement des données

In [ ]:
# Configuration
DATA_DIR = Path("../data")
MODEL_NAME = "resnet50"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32

print(f"Device: {DEVICE}")
print(f"Model: {MODEL_NAME}")

In [ ]:
# Initialisation des composants
preprocessor = ImagePreprocessor()
feature_extractor = FeatureExtractor(model_name=MODEL_NAME, device=DEVICE)
data_loader = DataLoaderWrapper(DATA_DIR)

# Découverte des images
image_paths = data_loader.discover_images()
print(f"Nombre d'images trouvées: {len(image_paths)}")

# Limiter à un sous-ensemble pour les tests
if len(image_paths) > 100:
    image_paths = image_paths[:100]
    print(f"Utilisation d'un sous-ensemble de {len(image_paths)} images pour les tests")

## 2. Extraction des features

In [ ]:
# Extraction des features
print("Début de l'extraction des features...")
features = feature_extractor.extract_features_from_paths(
    image_paths=image_paths,
    preprocessor=preprocessor,
    batch_size=BATCH_SIZE
)

print(f"Shape des features: {features.shape}")
print(f"Dimension des features: {features.shape[1]}")

# Sauvegarde des features
features_path = DATA_DIR / "features_resnet50.npy"
np.save(features_path, features)
print(f"Features sauvegardées dans: {features_path}")

## 3. Analyse statistique des features

In [ ]:
# Statistiques descriptives
features_df = pd.DataFrame(features)

print("\nStatistiques des features:")
print(features_df.describe().T[['mean', 'std', 'min', 'max']].head(10))

# Visualisation des distributions
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Distribution des premières dimensions
for i, ax in enumerate(axes.flatten()):
    if i < features.shape[1]:
        ax.hist(features[:, i], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        ax.set_title(f"Distribution de la feature {i}")
        ax.set_xlabel("Valeur")
        ax.set_ylabel("Fréquence")

plt.tight_layout()
plt.show()

## 4. Réduction de dimensionnalité pour visualisation

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# PCA pour réduction à 2D
pca = PCA(n_components=2, random_state=42)
features_pca = pca.fit_transform(features)

# t-SNE pour visualisation
tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(features)-1))
features_tsne = tsne.fit_transform(features)

print(f"Variance expliquée par PCA: {pca.explained_variance_ratio_.sum():.2%}")
print(f"Variance expliquée par composante 1: {pca.explained_variance_ratio_[0]:.2%}")
print(f"Variance expliquée par composante 2: {pca.explained_variance_ratio_[1]:.2%}")

In [ ]:
# Visualisation PCA
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(features_pca[:, 0], features_pca[:, 1], alpha=0.6, s=20)
plt.title("PCA des features ResNet50")
plt.xlabel("Composante principale 1")
plt.ylabel("Composante principale 2")
plt.grid(True, alpha=0.3)

# Visualisation t-SNE
plt.subplot(1, 2, 2)
plt.scatter(features_tsne[:, 0], features_tsne[:, 1], alpha=0.6, s=20)
plt.title("t-SNE des features ResNet50")
plt.xlabel("t-SNE dimension 1")
plt.ylabel("t-SNE dimension 2")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Analyse de similarité

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcul de la matrice de similarité cosinus
similarity_matrix = cosine_similarity(features)

# Visualisation de la matrice de similarité
plt.figure(figsize=(10, 8))
plt.imshow(similarity_matrix, cmap='viridis', aspect='auto')
plt.colorbar(label='Similarité cosinus')
plt.title('Matrice de similarité entre images')
plt.xlabel('Index image')
plt.ylabel('Index image')
plt.show()

# Statistiques de similarité
similarity_values = similarity_matrix[np.triu_indices_from(similarity_matrix, k=1)]
print(f"\nStatistiques de similarité:")
print(f"Moyenne: {similarity_values.mean():.3f}")
print(f"Écart-type: {similarity_values.std():.3f}")
print(f"Min: {similarity_values.min():.3f}")
print(f"Max: {similarity_values.max():.3f}")

## 6. Préparation pour le clustering

In [ ]:
# Sauvegarde des données pour le clustering
import pickle

clustering_data = {
    'features': features,
    'image_paths': [str(p) for p in image_paths],
    'features_pca': features_pca,
    'features_tsne': features_tsne,
    'similarity_matrix': similarity_matrix
}

clustering_path = DATA_DIR / "clustering_data.pkl"
with open(clustering_path, 'wb') as f:
    pickle.dump(clustering_data, f)

print(f"Données pour clustering sauvegardées dans: {clustering_path}")
print(f"Taille des features: {features.shape}")
print(f"Nombre d'images: {len(image_paths)}")

## 7. Résumé et prochaines étapes

In [ ]:
print("\n=== RÉSUMÉ DE L'EXTRACTION DE FEATURES ===")
print(f"Modèle utilisé: {MODEL_NAME}")
print(f"Nombre d'images traitées: {len(image_paths)}")
print(f"Dimension des features: {features.shape[1]}")
print(f"Variance expliquée par PCA (2D): {pca.explained_variance_ratio_.sum():.2%}")
print(f"Similarité moyenne entre images: {similarity_values.mean():.3f}")
print("\nProchaines étapes:")
print("1. Appliquer le clustering (K-Means, DBSCAN) sur les features")
print("2. Générer des labels faibles à partir des clusters")
print("3. Visualiser les clusters dans l'espace réduit")
print("4. Évaluer la qualité du clustering avec métriques de silhouette")

## 8. Tests unitaires intégrés

In [ ]:
# Tests de validation
def test_feature_extraction():
    """Test basique de l'extraction de features."""
    assert features.shape[0] == len(image_paths), "Nombre de features doit correspondre au nombre d'images"
    assert features.shape[1] == feature_extractor.feature_dim, "Dimension des features incorrecte"
    assert not np.any(np.isnan(features)), "Les features ne doivent pas contenir de NaN"
    assert not np.any(np.isinf(features)), "Les features ne doivent pas contenir d'infini"
    print("✅ Tests de validation des features réussis")

test_feature_extraction()